Only run this notebook if you have verified your ChromeDriver setup in the `test_*.ipynb` notebooks.

We are scraping everlane, which has implemented an infinite scroll system within their ecommerce site.

Infinite scroll means there is no pagination number within the page, and it will scroll as long as there is a next page. Because you can't see the end from where you look, you will have to apply a `while` loop to get it done.

In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import StaleElementReferenceException
from splinter.exceptions import ElementDoesNotExist

In [2]:
# Open a Chrome window using Splinter
browser = Browser('chrome')

The chromedriver version (123.0.6312.122) detected in PATH at C:\Users\jon_1\bin\chromedriver.exe might not be compatible with the detected chrome version (124.0.6367.119); currently, chromedriver 124.0.6367.155 is recommended for chrome 124.*, so it is advised to delete the driver in PATH and retry


In [3]:
# Setup the URL you want to scrape
url = "https://www.everlane.com/products/womens-organic-cotton-crew-sweater-canvas-black?collection=womens-best-sellers"

In [4]:
# Use the browser to visit the URL
browser.visit(url)

In [5]:
# Close popup
try:
    browser.find_by_text('No Thanks ').first.click()
except IndexError:
    pass
except ElementDoesNotExist:
    pass

In [6]:
# Gather the HTML from the current page in the browser
html = browser.html

In [7]:
html

'<html lang="en" class=""><head><meta charset="utf-8"><meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0,user-scalable=0"><link rel="icon" href="https://media.everlane.com/image/upload/c_scale/c_fill,dpr_auto,f_auto,q_auto,w_auto/v1/static/favicon4.ico" type="image/x-icon"><link rel="apple-touch-icon apple-touch-icon-precomposed" sizes="76x76" href="https://media.everlane.com/image/upload/c_fill,dpr_1,f_auto,g_face:center,h_76,q_auto,w_76/v1/static/1x_ipad.png"><link rel="apple-touch-icon apple-touch-icon-precomposed" sizes="167x167" href="https://media.everlane.com/image/upload/c_fill,dpr_1,f_auto,g_face:center,h_167,q_auto,w_167/v1/static/2x_ipad_pro.png"><link rel="apple-touch-icon apple-touch-icon-precomposed" sizes="152x152" href="https://media.everlane.com/image/upload/c_fill,dpr_1,f_auto,g_face:center,h_152,q_auto,w_152/v1/static/2x_ipad.png"><link rel="apple-touch-icon apple-touch-icon-precomposed" sizes="120x120" href="https://media.everlan

In [8]:
# Use Beautiful Soup to parse the HTML tags into objects.
soup = BeautifulSoup(html, 'html.parser')

In [9]:
rating_value_list = soup.find_all('meta', attrs={"itemprop": "ratingValue"})
rating_value_list

[<meta content="5" itemprop="ratingValue"/>,
 <meta content="5" itemprop="ratingValue"/>,
 <meta content="5" itemprop="ratingValue"/>,
 <meta content="5" itemprop="ratingValue"/>,
 <meta content="5" itemprop="ratingValue"/>,
 <meta content="5" itemprop="ratingValue"/>,
 <meta content="5" itemprop="ratingValue"/>,
 <meta content="5" itemprop="ratingValue"/>]

In [10]:
comments_list = soup.find_all('div', attrs={"class", "bv-content-summary-body-text"})
comments_list

[<div class="bv-content-summary-body-text"> <p>I have this in two colors. I have the grey in a medium which is really oversized, so I bought the second one in XS and it’s perfect. The M is longer and really wide, I like oversized sweater but since it’s on the thinner side it did not look as good. I am usually a S in everlane sweaters.</p> </div>,
 <div class="bv-content-summary-body-text"> <p>Fits on the larger side. Quality is high end, Soft with a lovely knit. Boxy fit. Just a perfect classy chic sweater to wear anywhere.</p> </div>,
 <div class="bv-content-summary-body-text"> <p>Beautyful cotton, warm yet not heavy. Fits oversized. I bought an M and could have sized down.</p> </div>,
 <div class="bv-content-summary-body-text"> <p>The grass green is incredible and vibrant and exactly what I wanted. It's a true kelly green--one of the photos makes it look more like a jade green, but it's not. I am obsessed with the colour. </p><p>The fit is baggy and boxy, with a nice weight to it--pe

#### Pagination

In [11]:
# Trigger link by finding the exact css to click on
browser.find_by_css('.bv-content-btn.bv-content-btn-pages.bv-content-btn-pages-load-more.bv-focusable').first.click()

In [13]:
# As we don't know when does the infinite scroll end, we need a while loop
more_bool = True

while more_bool:
    try: 
        # This is where you put in the logic to scrap the reviews section
        # I will not be including it as this is something you should tailor for you own use case.

        browser.find_by_css('.bv-content-btn.bv-content-btn-pages.bv-content-btn-pages-load-more.bv-focusable').first.click()
    except StaleElementReferenceException:
        more_bool = False
        pass
    except IndexError:
        more_bool = False
        pass
    except ElementDoesNotExist:
        more_bool = False
        pass


StaleElementReferenceException: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=124.0.6367.119); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF614437032+63090]
	(No symbol) [0x00007FF6143A2C82]
	(No symbol) [0x00007FF61423EC65]
	(No symbol) [0x00007FF614243D21]
	(No symbol) [0x00007FF614246159]
	(No symbol) [0x00007FF614246200]
	(No symbol) [0x00007FF6142867B5]
	(No symbol) [0x00007FF61427A496]
	(No symbol) [0x00007FF6142A6FDA]
	(No symbol) [0x00007FF61427A00A]
	(No symbol) [0x00007FF6142A71F0]
	(No symbol) [0x00007FF6142C3412]
	(No symbol) [0x00007FF6142A6D83]
	(No symbol) [0x00007FF6142783A8]
	(No symbol) [0x00007FF614279441]
	GetHandleVerifier [0x00007FF6148325AD+4238317]
	GetHandleVerifier [0x00007FF61486F70D+4488525]
	GetHandleVerifier [0x00007FF6148679EF+4456495]
	GetHandleVerifie

In [ ]:
browser.quit()